In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import random

In [2]:
with open('movie_lines.txt') as f:
    content = f.readlines()

In [3]:
def strip_line(line):
    sep = ' +++$+++ '    
    ind = line.rfind(sep) + len(sep)
    return (line[ind:]).strip().replace(u'\xa0', u' ')

In [4]:
result = []
for l in content:
    l = strip_line(l)
    if (len(l)<30) or '...' in l or ' . ' in l or '<u>' in l: continue
    if '--' in l or '\t\t' in l : continue
    sents = sent_tokenize(l)
    if len(sents) > 4 : continue
    result.append(sents)

In [5]:
result = random.sample(result, 500)

In [11]:
tech_crunch = pd.read_csv('./techcrunch_posts.csv')

In [12]:
posts = tech_crunch['content'].sample(600).tolist()

In [13]:
cropped_posts = []
for p in posts:
    p = str(p)
    ind = p.find('.', 2000)
    cropped_posts.append(p[:ind].replace(u'\xa0', u' '))

In [18]:
posts_sents = [sent_tokenize(p.strip()) for p in cropped_posts]

In [15]:
def get_short_span(sentences):
    start_ind = -1;
    for i in range(len(sentences)):
        if len(sentences[i]) <= 40:
            start_ind = i
            break
    if start_ind == -1 or start_ind == len(sentences) - 1:
        return sentences[:1]
    end_ind = -1
    for i in range(start_ind, len(sentences)):
        if len(sentences[i]) > 40:
            end_ind = i - 1
            break
    if end_ind == -1 or end_ind == start_ind:
        return sentences[start_ind:(start_ind + 2)]
    return sentences[start_ind:end_ind]

In [19]:
tech_results = [get_short_span(p) for p in posts_sents]

In [25]:
import spacy
nlp = spacy.load('en_core_web_md')

In [26]:
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [38]:
doc = nlp("Some text...")

In [39]:
for t in doc:
    print(t.text)

Some
text
...


In [65]:
def random_first_cap(token):
    if token.upper() == token:
        return token
    if random.randint(0,1) == 0:
        return token[0].lower() + token[1:]
    return token

In [86]:
def merge_seq(sents):
    result = []
    for i in range(len(sents)):
        s = sents[i]        
        doc = nlp(s)
        tokens = [t.text for t in doc]
        if len(sents) == 1:
            result = [[t,False] for t in tokens]
            break
        if i == 0:
            result += [[t,False] for t in tokens[:-2]]
            result.append([tokens[-2],True])
            continue
        if i == len(sents) - 1:
            result.append([random_first_cap(tokens[0]), False])
            result += [[t,False] for t in tokens[1:]]
            continue
        else:
            result.append([random_first_cap(tokens[0]), False])
            result += [[t,False] for t in tokens[1:-2]]
            result.append([tokens[-2],True])
    return result

[['Where', False],
 ['are', False],
 ['you', False],
 ['going', True],
 ['you', False],
 ["'ll", False],
 ['freeze', False],
 ['out', False],
 ['there', True],
 ['You', False],
 ['do', False],
 ["n't", False],
 ['even', False],
 ['have', False],
 ['a', False],
 ['coat', False],
 ['.', False]]

In [88]:
import json
data = []
for sents in result+tech_results:
    data.append(merge_seq(sents))
with open('train_data.json', 'w') as outfile:
    json.dump(data, outfile)